BerkeleyX: Data8.3x

Foundations of Data Science: Prediction and Machine Learning

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import patches
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

Lab 3: Regression Inference

1: The Age of the Universe

In [ ]:
num_locations = 15
example_velocities = Table().with_columns(
    "x", np.random.normal(size=num_locations),
    "y", np.random.normal(size=num_locations))
start_of_time = -2

def scatter_after_time(t, start_of_time, end_of_time, velocities, center_name, other_point_name, make_title):
    max_location = 1.1*(end_of_time-start_of_time)*max(max(abs(velocities.column("x"))), max(abs(velocities.column("y"))))
    new_locations = velocities.with_columns(
            "x", (t-start_of_time)*velocities.column("x"),
            "y", (t-start_of_time)*velocities.column("y"))
    plt.scatter(make_array(0), make_array(0), label=center_name, s=100, c="yellow")
    plt.scatter(new_locations.column("x"), new_locations.column("y"), label=other_point_name)
    for i in np.arange(new_locations.num_rows):
        plt.arrow(
            new_locations.column("x").item(i),
            new_locations.column("y").item(i),
            velocities.column("x").item(i),
            velocities.column("y").item(i),
            fc='black',
            ec='black',
            head_width=0.025*max_location,
            lw=.15)
    plt.xlim(-max_location, max_location)
    plt.ylim(-max_location, max_location)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_position(make_array(0, 0, 1, 1))
    plt.legend(bbox_to_anchor=(1.6, .7))
    plt.title(make_title(t))
    plt.show()

interact(
    scatter_after_time,
    t = widgets.FloatSlider(min=start_of_time, max=5, step=.05, value=0, msg_throttle=1),
    start_of_time = fixed(start_of_time),
    end_of_time = fixed(5),
    velocities = fixed(example_velocities),
    center_name = fixed("our sun"),
    other_point_name = fixed("other star"),
    make_title = fixed(
        lambda t: "The world {:01g} year{} in the {}".format(
            abs(t),"" if abs(t) == 1 else "s", "past" if t < 0 else "future")))

Analogy: driving

In [ ]:
mei_velocity = Table().with_columns("x", make_array(60), "y", make_array(0))

interact(
    scatter_after_time,
    t = widgets.FloatSlider(min=-2, max=1, step=.05, value=0, msg_throttle=1),
    start_of_time = fixed(-2),
    end_of_time = fixed(1),
    velocities = fixed(mei_velocity),
    center_name = fixed("Us"),
    other_point_name = fixed("Mei"),
    make_title = fixed(
        lambda t: "Mei's position {:01g} hour{} in the {}".format(
            abs(t), "" if abs(t) == 1 else "s", "past" if t < 0 else "future")))

In [ ]:
small_driving_example = Table().with_columns(
        "Name",                                       make_array("Us", "Mei"),
        "Speed moving away from us (miles per hour)", make_array(0,    60),
        "Current distance from us (miles)",           make_array(0,    120))

small_driving_example.scatter(1, 2, s=200, fit_line=True)

# Fancy magic to draw each person's name with their dot.
with_slope_indicator = small_driving_example.with_row(
    ["Slope = 2\ hours", small_driving_example.column(1).mean(), small_driving_example.column(2).mean()])
for i in range(with_slope_indicator.num_rows):
    name = with_slope_indicator.column(0).item(i)
    x = with_slope_indicator.column(1).item(i)
    y = with_slope_indicator.column(2).item(i)
    plt.scatter(make_array(x - 15), make_array(y + 15), s=1000*len(name), marker="$\mathrm{" + name + "}$")

In [ ]:
Table.read_table('../../data/drivers.csv').scatter(0, 1, fit_line=True)

In [ ]:
# Question 1.1 By looking at the fit line, estimate how long ago (in hours) Mei left.
driving_start_time_hours = 2

Back to cosmology

In [ ]:
close_novas = Table.read_table("../../data/close_novas.csv")
close_novae = close_novas

close_novas.scatter(0, 1, fit_line=True)
close_novas.show(5)

In [ ]:
# Question 1.2 Looking this plot, make a guess at the age of the universe
first_guess_universe_age_years = 20e9

Fitting the line yourself

Question 1.3

errors() takes three arguments:
1. a table like `close_novas`
2. the slope of a line
3. the intercept of a line

It returns an array of the errors made when a line with that slope and intercept is used to predict distance from speed for each supernova in the given table.  (The error is the actual distance minus the predicted distance.)

In [ ]:
def errors(t, slope, intercept):
    return t.column(1) - (slope * t.column(0) + intercept)

In [ ]:
er = errors(close_novas, 16_000, 0)
novas_plus = close_novas.with_columns('er', er)
novas_plus.scatter(0)

Question 1.4

Using errors, compute the errors for the line with slope 16000 and intercept 0 on the close_novas dataset.

Then make a scatter plot of the errors.

In [ ]:
example_errors = errors(close_novas, 16_000, 0)
novas_plus = close_novas.with_column('error', example_errors)
novas_plus.scatter(0, 2)

Question 1.5 fit_line()

A table like close_novas as its argument.

Return an array containing the slope (as item 0) and intercept (as item 1) of the least-squares regression line predicting distance from speed for that table.

In [ ]:
def standard_units(any_numbers):
    """Convert any array of numbers to standard units."""
    return (any_numbers - np.average(any_numbers)) / np.std(any_numbers)

def correlation(t, x, y):
    """Return the correlation coefficient (r) of two variables."""
    return np.mean(standard_units(t.column(x)) * standard_units(t.column(y)))

def slope(t, x, y):
    """The slope of the regression line (original units)."""
    r = correlation(t, x, y)
    return r * np.std(t.column(y)) / np.std(t.column(x))

def intercept(t, x, y):
    """The intercept of the regression line (original units)."""
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))

def fit_line(tbl):
    return make_array(slope(tbl, 0, 1), intercept(tbl, 0, 1))

In [ ]:
example_table = Table().with_columns(
    "Speed (parsecs/year)", make_array(0, 1),
    "Distance (million parsecs)", make_array(1, 3))
fit_line(example_table)

Question 1.6 Use your function to fit a line to close_novas.

Then, set new_errors equal to the errors that we get calling errors with our new line

In [ ]:
fit_line(close_novas)

In [ ]:
best_line_slope, best_line_intercept = fit_line(close_novas)

new_errors = errors(close_novas, best_line_slope, best_line_intercept)

In [ ]:
# This code displays the residual plot, given your values for the best_line_slope and best_line_intercept
Table().with_column("Speed (parsecs/year)", 
                    close_novas.column("Speed (parsecs/year)"), 
                    "Distance errors (million parsecs)", 
                    new_errors
                   ).scatter(0, 1, fit_line=True)

# This just shows your answer as a nice string, in billions of years.
"Slope: {:g} (corresponding to an estimated age of {:,} billion years)".format(best_line_slope, round(best_line_slope/1000, 4))

Question 1.7 Simulate 1000 resamples from close_novas

In [ ]:
counter = 1_000
bootstrap_ages = np.empty(counter)
for i in range(counter):
        resample = close_novas.sample()
        bootstrap_ages[i] = 1_000_000 * slope(resample, 0, 1)

lower_end = percentile(2.5, bootstrap_ages)
upper_end = percentile(97.5, bootstrap_ages)
Table().with_column("Age estimate", bootstrap_ages*1e-9).hist(
    bins=np.arange(12, 16, .1), unit="billion years", normed=None, density=True)
print("95% confidence interval for the age of the universe: [{:g}, {:g}] billion years".format(lower_end*1e-9, upper_end*1e-9))